# Attacking Historical Ciphers

There's a variety of techniques we can use to attack historical ciphers. We're going to take a look at some of these techniques in the scope of these simpler, historical ciphers first.

We'll start with the **Atbash cipher**.

We're also going to include the simple_ciphers module that we created to encapsulate the historical ciphers from the first notebook.

In [ ]:
import simple_ciphers as sc

letters = list(map(chr, range(65, 91)))
letters.append(' ')

atbash = sc.Atbash(alphabet=letters)

If we have the ciphertext and the plaintext, substitution ciphers are trivial to break.

In [ ]:
message = 'The brown fox jumped over the lazy dog'
ciphertext = atbash.encipher(plaintext=message)
message = message.upper()

decryption_key = {}
for idx, c in enumerate(message):
    decryption_key[c] = ciphertext[idx]

key_list = list(decryption_key.keys())
key_list.sort()

print('Partial Encryption Transformation')
for key in key_list:
    print('{} -> {}'.format(key, decryption_key[key]))

Looking at this, it's pretty clear we have an Atbash cipher.

What if we don't have plaintext though? what then? Well, we can still attack the cipher using statistics. Specifically, we take the ciphertext, generate a random key, and attempt to use that key to decipher the ciphertext. We then use a fitness measure to determine how close we are to generating actual words. We repeat this until we seem to have a reasonable score. This is the **hillclimbing algorithm**.

In [ ]:
import random as r
import math as m
import ngram as ng

class HillClimbing:
    
    def _mutate(self, decryption_key):
        keys = list(decryption_key.keys())
        idx_0, idx_1 = r.randint(0, len(keys) - 1), r.randint(0, len(keys) - 1)
        temp = decryption_key[keys[idx_0]]
        decryption_key[keys[idx_0]] = decryption_key[keys[idx_1]]
        decryption_key[keys[idx_1]] = temp
        return decryption_key
    
    def _generate_random_key(self, alphabet):
        parent = alphabet.copy()
        r.shuffle(parent, r.random)
        return sc.create_decryption_key(symbols=alphabet, key_symbols=parent)
        
    def __call__(self, alphabet, ciphertext):
        
        # Defining the best scores.
        best_score = {'score': -99e9, 'key': None, 'plaintext': None}
        
        while True:
            # Generate the random key.
            potential_key = self._generate_random_key(alphabet)

            # Attempt to decipher the ciphertext.
            plaintext = sc.decipher(message=ciphertext, decryption_key=potential_key)

            # Initialize the N-Gram scoring class.
            evaluator = ng.ngram_score('english_quadgrams.txt')

            # Evaluate the ciphertext score.
            score = evaluator(plaintext)

            # Save the current best loop key and score
            loop_score = {'score': score, 'key': potential_key, 'plaintext': plaintext}

            cnt = 0
            while cnt < 1000:
                # Mutate the key
                potential_key = self._mutate(decryption_key=potential_key)

                # Attempt to decipher the ciphertext.
                plaintext = sc.decipher(message=ciphertext, decryption_key=potential_key)

                # Evaluate the ciphertext score.
                score = evaluator(plaintext)

                # Did we beat our best? then save it!
                if score > loop_score['score']:
                    cnt = 0
                    loop_score['score'] = score
                    loop_score['key'] = potential_key
                    loop_score['plaintext'] = plaintext
                    
                # Increment the counter.
                cnt += 1
                
            if loop_score['score'] > best_score['score']:
                best_score['score'] = loop_score['score']
                best_score['key'] = loop_score['key']
                best_score['plaintext'] = loop_score['plaintext']
                print('({}) {}'.format(best_score['score'], best_score['plaintext']))

message = 'The brown fox jumped over the lazy dog'
ciphertext = atbash.encipher(plaintext=message)
key = HillClimbing()(
    alphabet=letters, 
    ciphertext=ciphertext
)

You might have noticed, you need to explicitly stop the algorithm by pressing the stop button at the top of the notebook. The hillclimbing algorithm works, but it can take FOREVER. I suggest you copy this out of this notebook and run it as a script if you want to see it in action. Our case is a bit harder because we added a space (i.e. ' ') to our alphabet too, so it can be more difficult to see the plaintext start to emerge.